In [4]:
import os
import sys
sys.dont_write_bytecode = True
sys.path.append('../tools/')
from maneger import DataManager
# from tools import preproc
from preproc import Preprocessor
# from utterance.feature import Featur
import spacy
# import importlib
pre = Preprocessor()
import pprint
import random

import copy
from feature import Feature
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics


300


In [5]:
nlp = spacy.load('ja_ginza')

In [6]:
def get_POS(texts):
    
    pos_list = []
    docs = list(nlp.pipe(texts, disable=['ner']))
    for doc in docs:
        pos_list.append([ token.tag_ for token in doc ])
        
    return pos_list

In [7]:
def div2sentence(text):
    if isinstance(text, str):
        doc = nlp(text)
        sentence_list = [str(s)  for s in doc.sents]

    elif isinstance(text, list):
        sentence_list  = []
        docs = list(nlp.pipe(text, disable=['ner']))
            # return [ self.get_POS(sen_) for sen_ in sen]
        for doc in docs:
            sentence_list.extend( [str(s) for s in doc.sents] )
    else:
        return None
    
    return sentence_list 

In [8]:
def make_token_set(texts):
    token_set = set()
    docs = list(nlp.pipe(texts, disable=['ner']))
    for doc in docs:
        for token in doc:
            token_set.add(token.text)
    return token_set

In [9]:
def get_ave_length(texts):
    docs = list(nlp.pipe(texts, disable=['ner']))
    ave_length = 0
    for doc in docs:
        ave_length += len(doc)
    ave_length = int(ave_length/len(docs)) + 1
    # ave_length = ave_length/len(doc)
    return ave_length

In [10]:
def make_random_sentence(token_list, length):
    samples = random.choices(token_list, k=length)
    return "".join(samples)

In [11]:
def make_feature(sentence_list):
    features = []
    docs = list(nlp.pipe(sentence_list, disable=['ner']))
    # 名詞
    # for doc in docs:
    



In [12]:
root = "/".join(os.getcwd().split("/")[:-1])
root

'/home/yamada/Documents/MMI/UNI'

In [13]:
path = root + '/error_category_classification/dbdc5_ja_dev_labeled/'
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']


In [14]:
df = pre.read_json_with_NoErr(path, datalist)
print(df.shape)

(2000, 5)


In [15]:
vec_mode = "ginza"
data_mode_list = ["dialogue", "wiki"]
data_mode = data_mode_list[0]
data_path = "./X_y_data/{0}/".format(data_mode)
dataM = DataManager(data_path)
print(data_path)
way = ["LR", "DNN"]
data_name = "data_{0}.pickle".format(way[0])
print(data_name)

model_path = "./models/{0}/".format(data_mode)
modelM = DataManager(model_path)


./X_y_data/dialogue/
data_LR.pickle


In [22]:
if dataM.is_exist(data_name):
    
    DATA_Xy = dataM.load_data(data_name)
    X_data = DATA_Xy[0]
    y_data = DATA_Xy[1]
else:
    X_data, y_data = pre.extract_X_y(df, error_types, seq_len)
    dataM.save_data(data_name, [X_data, y_data])

NameError: name 'seq_len' is not defined

In [17]:
# ユーザ発話のみ取得し，操作

usr_list = []
for d, u, s, ec in zip(df.did, df.usr, df.sys, df.ec):
    usr_list.append(u)

In [18]:

# text = copy.deepcopy(usr_list[:20])
text = copy.deepcopy(usr_list)
text = div2sentence(text)
# pprint.pprint(text[:20])


In [19]:

token_set = make_token_set(text)
token_list = list(token_set)
ave_length = get_ave_length(text)

correct_len = len(text)
X_str = copy.deepcopy(text)
y = [1]*correct_len
for _ in range(correct_len):
    X_str.append( make_random_sentence(token_list, ave_length) )
    y.append(0)
y = np.array(y)


In [20]:

X_train_str, X_test_str, y_train, y_test = train_test_split(X_str, y, test_size=0.30, random_state=5)

In [23]:
F = Feature()
F.set_preprocessor(Preprocessor())
F.make_features(X_train_str)

F_path = "../X_y_data/utterance/"
F_name = "grammer.pickle"
featureM = DataManager(F_path)

300


In [24]:
featureM.save_data(F_name, F)

success save : ../X_y_data/utterance/grammer.pickle


In [25]:
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [26]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, solver='sag')

In [27]:
y_pred = lr.predict(X_test)

In [28]:
for y_p, x_s in zip(y_pred[:10], X_test_str[:10]):
    print("{0} : {1}".format(y_p, x_s))

1 : 朝ニッカウヰスキーでしょうそこお気に入りシンガポールって家
1 : 初めて聞いた名前
0 : ブラック教わり頑張りこっち」、カッコイイお
0 : 危険ジーニストエコ日本食椅子持ち味技術メロン
1 : 私は1位じゃないことに驚きました
1 : 私は喜多方ラーメンが好きです
1 : 入社前の裁判は日本テレビですよ。
1 : ミラノ版なんてあるんだね
0 : 同じくネイマール国際賢くスイス大リーグかっこいい歴史
0 : 反応ダイエットラーメン適度褒めヒカル矢部うどん


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[734  16]
 [  4 728]]
accuracy =  0.9865047233468286
precision =  0.978494623655914
recall =  0.994535519125683
f1 score =  0.986449864498645


In [30]:
test = ["これはペンです", "F1スコアは、適合率と再現率の調和平均で定義されます", "鉛筆は下書きを描くのです"]
XX = []
for t in test:
    XX.append(F.featurization(t))
pred =  lr.predict(XX)

In [31]:
pred

array([1, 1, 1])

In [32]:
model_path = "../models/utterance/"
model_name = "grammer_M.pickle"
modelM = DataManager(model_path)
print(model_name)

grammer_M.pickle


In [33]:
modelM.save_data(model_name, lr)

success save : ../models/utterance/grammer_M.pickle
